In [ ]:
import numpy as np
import twitter
import matplotlib.pyplot as plt
import json

CONSUMER_KEY = 'fjRSNYpRexhGGEBRZ6wDtR46W' #API key
CONSUMER_SECRET = 'TsN5f7qN6SHUFliBptqqu4FFh3YLx6tisldkvFx5C065HiwaoE'#secret api key
OAUTH_TOKEN = '804135069988306947-WoNcUYrqIJsfcdKegraukBv3oF4hsKA' #OAUTH1 token
OAUTH_TOKEN_SECRET = 'x3O77abe4vDaQ40ad8h5SforRGoLY7G9kBjOiRuTBrnEj' #OAUTH1 secret token

username = "edmundyu1001"

### BEARER_TOKEN: AAAAAAAAAAAAAAAAAAAAAMp2ZwEAAAAAx1FMkvOgtdik4B0KKqoD3enIKZk%3DjLbLeDoYvSJTFcNzyzpKHsMytYx8htHWn14w6zy5XPDC32lxLR


auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET, CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

print("twitter_api: ", twitter_api)


In [ ]:
# retrieve friends from a user
import cookbook as cb

# friends_ids, followers_ids = cb.get_friends_followers_ids(twitter_api, screen_name=username)

# #get all reciprocal friends and put in a list
# reciprocal_friends = set(friends_ids) & set(followers_ids)

# #turn the set into a list so that it can be sliced
# l_recip = list(reciprocal_friends)

# first_100 = l_recip[:100]

# followers_with_count = []

# i = 0
# five_most_popular = []
# while i < len(l_recip):
#     friend_batch = l_recip[i:i+100]
#     print("friend_batch: ", len(friend_batch))
#     response = cb.get_user_profile(twitter_api, user_ids=friend_batch)
#     for r in response:
#         followers_with_count.append((response[r]["id"],response[r]["followers_count"]))
#     i += 100


# def find_five_pop_reciprocals(friends_ids, followers_ids):
#     reciprocal_friends = set(friends_ids) & set(followers_ids)
#     l_recip = list(reciprocal_friends)

#     first_100 = l_recip[:100]

#     followers_with_count = []

#     i = 0
#     five_most_popular = []
#     while i < len(l_recip):
#         friend_batch = l_recip[i:i+100]
#         print("friend_batch: ", len(friend_batch))
#         response = cb.get_user_profile(twitter_api, user_ids=friend_batch)
#         for r in response:
#             followers_with_count.append((response[r]["id"],response[r]["followers_count"]))
#         i += 100

#     followers_with_count.sort(key=lambda x: x[1],reverse=True)

#     top_five = followers_with_count[:5]
#     print("top five: ", top_five)

#     fmp_user_ids = [i[0] for i in top_five]

#     return fmp_user_ids

In [ ]:
import cookbook as cb
import networkx as nx 
# cb.crawl_followers(twitter_api, username, limit=10, depth=2)

depth = 2
limit =4


seed_id = str(twitter_api.users.show(screen_name=username)['id'])
print("seed id: ", seed_id)

#get the first users friends and followers
friend_queue, follower_queue = cb.get_friends_followers_ids(twitter_api, user_id=seed_id, 
                                          friends_limit=limit, followers_limit=limit)

print("follower queue of YU: ", follower_queue)
"""
TODO: find the reciprocal friends, as done in a2 file cell 2 -------------------------------------------- DONE
TODO: find the 5 most popular reciprocal friends, as done in a2 file ------------------------------------ DONE
TODO: create networkx graph, and make the connections between reciprocal friends, and the user
TODO: calculate diameter of networkx graph
TODO: calculate average distance of networkx graph
"""
print("five most popular........")
five_most_popular = find_five_pop_reciprocals(friend_queue, follower_queue)
for x in five_most_popular:
    print(x)

fmp_user_ids = people = [i[0] for i in five_most_popular]
print("FMP: ", fmp_user_ids)

d = 1
print("entering loop...................")
while d < depth:
    d += 1
    (queue, follower_queue) = (follower_queue, [])
    for fid in queue:
        friend_ids, follower_ids = cb.get_friends_followers_ids(twitter_api, user_id=fid, 
                                                  friends_limit=limit, 
                                                  followers_limit=limit)

        # from these friends and folowers get the top 5 reciprocal and add to the follower queue

        fid_top_five = find_five_pop_reciprocals(friend_ids, follower_ids)

        print("adding to follower queue........................")
        follower_queue += fid_top_five
        print("len follower queue: ", len(follower_queue))
        friend_queue += fid_top_five

print("the follower queue finished at length:  ", len(follower_queue))


In [30]:
jfk = cb.make_twitter_request(twitter_api.followers.ids, screen_name=username, count = 5000)
iop = cb.make_twitter_request(twitter_api.friends.ids, screen_name=username, count=5000)

In [35]:
import networkx as nx
import matplotlib.pyplot as plt

def find_five_pop_reciprocals(friends_ids, followers_ids):
    reciprocal_friends = set(friends_ids) & set(followers_ids)
    l_recip = list(reciprocal_friends)

    first_100 = l_recip[:100]

    followers_with_count = []

    i = 0
    five_most_popular = []
    while i < len(l_recip):
        friend_batch = l_recip[i:i+100]
        print("friend_batch: ", len(friend_batch))
        response = cb.get_user_profile(twitter_api, user_ids=friend_batch)
        for r in response:
            followers_with_count.append((response[r]["id"],response[r]["followers_count"]))
        i += 100

    followers_with_count.sort(key=lambda x: x[1],reverse=True)

    top_five = followers_with_count[:5]
    print("top five: ", top_five)

    fmp_user_ids = [i[0] for i in top_five]

    return fmp_user_ids


G = nx.Graph()
ids = [165035772]

key = 165035772

G.add_nodes_from(ids)
# get all the friends and followers
# response = cb.make_twitter_request(twitter_api.followers.ids, screen_name=username, count = 5000)
# response1 = cb.make_twitter_request(twitter_api.friends.ids, screen_name=username, count=5000)

# next_queue = response["ids"]

# pull out just the ids fo the friends and followers
# followers = response["ids"]
# friends = response1["ids"]

followers = jfk["ids"]
friends = iop["ids"]

# find the 5 most popular reciprocals of the user
top_five_recip = find_five_pop_reciprocals(friends, followers)

print("top five recip: ", top_five_recip)


# print("Got followers for {0}: {1}".format(username,next_queue))

# ad the top five reciprocals to the ids list, going to do DFS on those until we have 100 ids
ids += top_five_recip

next_queue = top_five_recip


G.add_nodes_from(top_five_recip)

edges = map(lambda e: (key, e), top_five_recip)
G.add_edges_from(edges)




depth = 1
max_depth = 4

while depth < max_depth and len(ids) < 10:
  depth += 1
  (queue, next_queue) = (next_queue,[])

  for id in queue:
    response_a = cb.make_twitter_request(twitter_api.followers.ids, user_id=id, count = 5000) #change count to 5000 for real one
    response_b = cb.make_twitter_request(twitter_api.friends.ids, user_id=id, count=5000)
    followers = response_a["ids"]
    friends = response_a["ids"]
    top_five_recip = find_five_pop_reciprocals(friends, followers)
    print("top five recip: ", top_five_recip)  

    G.add_nodes_from(top_five_recip)

    edges = map(lambda e: (id, e), top_five_recip)
    G.add_edges_from(edges)

    if top_five_recip:
      print("Got followers for {0}: {1}".format(id,top_five_recip))
      for i in top_five_recip:
        if(i not in next_queue and i not in ids): #might not need the i not in ids part, probably not
          next_queue.append(i)
    else:
      print(str(id) + "is protected")

  ids += next_queue

print(ids)

nx.draw(G)
plt.savefig("mygraph.png")
plt.show()


friend_batch:  91
top five:  [(17881816, 2223035), (1933042237, 1855585), (14645160, 1574394), (76966092, 1271501), (5514192, 907636)]
top five recip:  [17881816, 1933042237, 14645160, 76966092, 5514192]
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100
friend_batch:  100


KeyboardInterrupt: 